# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Выбор локации для скважины

* Задача данного исследования - разобраться, где наиболее прибыльно будет бурить новую скважину. Это нам потребуется для более четкого распределения бюджета на локации с большим доходом от них. Для анализа был предоставлены датасеты со скважинами регионов.
* План действий:
    * Получить датасет
    * Обучить модель
    * Оценить качество моделей и выполнить расчеты
    * Воспользоваться техникой Bootstrap

## Загрузка и подготовка данных

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LinearRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [14]:
first_reg = pd.read_csv('../datasets/geo_data_0.csv')
second_reg = pd.read_csv('../datasets/geo_data_1.csv')
third_reg = pd.read_csv('../datasets/geo_data_2.csv')

In [15]:
def information(data):
    data.info()
    display(data.head(20))
    display(data.describe())

In [16]:
information(first_reg)
information(second_reg)
information(third_reg)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


По предварительному осмотру датасетов - они в порядке. Пропусков в данных нет, так что заполнять ничего не придется.

## Обучение и проверка модели

Первым делом разобьем датасеты на целевой и нецелевые признаки.

In [17]:
first_reg_features = first_reg.drop('product', axis=1)
second_reg_features = second_reg.drop('product', axis=1)
third_reg_features = third_reg.drop('product', axis=1)

first_reg_target = first_reg['product']
second_reg_target = second_reg['product']
third_reg_target = third_reg['product']

Напишем функцию, в которой обучаем модели линейной регрессией, потому что достаточно предсказуема, а также считаем среднеквадратическую ошибку наших моделей.

In [18]:
def ml_and_checking(features, target, region):
    features = features.drop('id', axis=1)
    scaler = StandardScaler()
    numerics = ['f0', 'f1', 'f2']
    scaler.fit_transform(features[numerics])
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted = model.predict(features_valid)
    print(f'Средний запас предсказанного сырья {region}:', predicted.mean())
    print(f'RMSE модели {region}:', mean_squared_error(y_true=target_valid, y_pred=predicted, squared=False))
    print()
    return pd.Series(predicted)

In [19]:
predicted_first_reg = ml_and_checking(first_reg_features, first_reg_target, 'first_region')
predicted_second_reg = ml_and_checking(second_reg_features, second_reg_target, 'second_region')
predicted_third_reg = ml_and_checking(third_reg_features, third_reg_target, 'third_region')

Средний запас предсказанного сырья first_region: 92.59256778438035
RMSE модели first_region: 37.5794217150813

Средний запас предсказанного сырья second_region: 68.728546895446
RMSE модели second_region: 0.8930992867756169

Средний запас предсказанного сырья third_region: 94.96504596800492
RMSE модели third_region: 40.02970873393434



Можно заметить, что лучше всего себя модель показала на втором регионе, *RMSE* составляет всего 0.89! Однако средний запас лежит в третьем регионе - 94.96.

## Подготовка к расчёту прибыли

Заведем в переменные имеющиеся у нас значения общего бюджета, количества скважин и доход с единицы продукта.

In [20]:
TOTAL_BUDGET = 10000000000
EXPLORE_REGION = 200
SELECTING_BOREHOLES = 500
INCOME_FROM_EACH_UNIT = 450000
NEEDED_VOLUME = TOTAL_BUDGET / EXPLORE_REGION / INCOME_FROM_EACH_UNIT
NEEDED_VOLUME

111.11111111111111

А также создадим функцию подсчета прибыли со скважин.

In [21]:
def revenue(volume):
    return volume * INCOME_FROM_EACH_UNIT - TOTAL_BUDGET

## Расчёт прибыли и рисков

Создадим функцию отбора двухста лучших скважин.

In [22]:
def best_boreholes(predicted_boreholes):
    top_200_boreholes = predicted_boreholes.sort_values(ascending=False, ignore_index=True).head(200)
    return top_200_boreholes


И теперь самое главное - реализуем технику *Bootstrap*, где будем 1000 раз создавать выборки из 500 скважин, выбирать из них 200 лучших и считать 95% доверительный интервал для прибыли

In [23]:
def bootstrap(predicted_boreholes, region):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        subsample = predicted_boreholes.sample(n=SELECTING_BOREHOLES, replace=True, random_state=state)
        subsample = best_boreholes(subsample)
        profit = revenue(subsample.sum())
        values.append(profit)
    values = pd.Series(values)
    mean_profit = values.mean()
    print(f'ДЛЯ РЕГИОНА {region.upper()}')
    print(' Средняя прибыль:', mean_profit / 1000000, 'млн')
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print(' Нижняя граница:', lower / 1000000, 'млн')
    print(' Верхняя граница:', upper/ 1000000, 'млн')
    print(f'  Процент риска: {(len(values[values < 0]) / len(values)):.2%}')




In [24]:
bootstrap(predicted_first_reg, 'first')
bootstrap(predicted_second_reg, 'second')
bootstrap(predicted_third_reg, 'third')


ДЛЯ РЕГИОНА FIRST
 Средняя прибыль: 358.02605420834124 млн
 Нижняя граница: 140.13733598107066 млн
 Верхняя граница: 598.8772713793849 млн
  Процент риска: 0.10%
ДЛЯ РЕГИОНА SECOND
 Средняя прибыль: 453.8116099461312 млн
 Нижняя граница: 32.88301962251606 млн
 Верхняя граница: 854.5616852130576 млн
  Процент риска: 1.60%
ДЛЯ РЕГИОНА THIRD
 Средняя прибыль: 281.46619794109176 млн
 Нижняя граница: 92.09830189543943 млн
 Верхняя граница: 478.5270324733727 млн
  Процент риска: 0.20%


Самая большая средня прибыль во втором регионе - 453 млн, однако и риск там самый высокий - 1.6%!

Таким образом, если нам хочется минимизировать риск убытков, то следует выбирать первым регион, так как шанс убытков там составляет 0.10% при средней прибыли 358 млн. Однако самая высокая средняя прибыль во втором регионе - 453 млн и риск составляет 1.6%, что меньше 2.5% по условию.

# Вывод

В данном проекте была проведена работа по получению данных, получению информации о них и дальнейшее использование в обучении моделей линейной регрессии. Модели были обучены успешно, особенно на данных второго региона, RMSE там составляет всего 0.89! Была посчитана необходимый обьем скважин для получения прибыль, а также создана функция расчета прибыли. Реализована техника *Boostrap*, в которой мы 1000 раз создавали выборки из 500 скважин и считали необходимые метрики. Таким образом наш глаз упал на второй регион, ведь именно там самая высокая средняя прибыль при риске ниже 2.5%!

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован